In [1]:
import os
import time
from IPython.display import clear_output

In [2]:
def time_consume(start_time):
    time_flow = time.time() - start_time
    hr_, min_, sec_ = 0, 0, 0
    if time_flow >= 3600:
        hr_ = time_flow // 3600
    if time_flow >= 60:
        min_ = (time_flow % 3600) // 60
    sec_ = time_flow % 60
    return "%s hr %s min %s sec" % (hr_, min_, sec_)

In [3]:
rna_seq_folder = "/media/sf_j1/MAP/RNA-seq/analysis"
if not os.path.isdir(rna_seq_folder + "/cufflinks"):
    os.makedirs(rna_seq_folder + "/alignment")
    os.makedirs(rna_seq_folder + "/cuffdiff")
    os.makedirs(rna_seq_folder + "/cufflinks")
    os.makedirs(rna_seq_folder + "/reference")
    os.makedirs(rna_seq_folder + "/gff")
    os.makedirs(rna_seq_folder + "/statistic")

In [4]:
#program directories
cufflinks = "/home/j1lim/다운로드/cufflinks/cufflinks"
cuffdiff = "/home/j1lim/다운로드/cufflinks/cuffdiff"
makegff = "/home/j1lim/다운로드/makegff.py"
samtools = "/home/j1lim/다운로드/samtools-1.9/samtools"

In [5]:
temp_lst = []
fastq_dir = "/home/j1lim/sbl_nas/MAP/RNA-seq/sequencing_result/fastq/"
fastq_files = os.listdir(fastq_dir)
fastq_files.sort()
for x in fastq_files:
    if "ATCC" not in x:
        continue
    temp_lst.append(x)
fastq_files = temp_lst
print fastq_files

['ATCC19688-1_1.fastq', 'ATCC19688-1_2.fastq', 'ATCC19688-2_1.fastq', 'ATCC19688-2_2.fastq']


In [6]:
#reference file directories
reference_file = "/media/sf_j1/MAP/RNA-seq/analysis/reference/GCF_000007865.1_ASM786v1_genomic.fna"
gff_file = "/media/sf_j1/MAP/RNA-seq/analysis/reference/GCF_000007865.1_ASM786v1_genomic.gff"

chromosome_size = "6000000" #bigger than larggest chromosome

In [7]:
#bowtie-build
indexed_file = rna_seq_folder + "/reference/" + reference_file.split("/")[-1]
bowtie_build_command = "bowtie-build -f %s %s" % (reference_file, indexed_file)
print bowtie_build_command

start_time = time.time()
#!$bowtie_build_command
print "\n* bowtie-built time consume = ", time_consume(start_time)

bowtie-build -f /media/sf_j1/MAP/RNA-seq/analysis/reference/GCF_000007865.1_ASM786v1_genomic.fna /media/sf_j1/MAP/RNA-seq/analysis/reference/GCF_000007865.1_ASM786v1_genomic.fna

* bowtie-built time consume =  0 hr 0 min 0.000128030776978 sec


In [8]:
for i in range(len(fastq_files)):
    if i % 2 == 1:
        continue
    #clear_output()
    fastq_left = fastq_dir + fastq_files[i]
    fastq_right = fastq_dir + fastq_files[i+1]
    print "%s & %s is now analyzing...\n" % (fastq_left, fastq_right)
    sample_name = fastq_files[i].split("_")[0]
    
    #bowtie
    sam_file = rna_seq_folder + "/alignment/%s.sam" % sample_name
    unaligned = rna_seq_folder + "/unaligned/unaligned_%s" % sample_name
    bowtie_command = "bowtie -X 1000 -n 2 -p 4 -3 3 -S %s -1 %s -2 %s > %s --un %s" % (
                    indexed_file, fastq_left, fastq_right, sam_file, unaligned)
    print bowtie_command, "\n"
    start_time = time.time()
    !$bowtie_command
    print "\n* bowtie time consume = ", time_consume(start_time), "\n"

    #samtools
    unsorted_bam_file = sam_file.replace(".sam", ".unsorted.bam")
    samtools_command = "%s view -bS %s -o %s" % (samtools, sam_file, unsorted_bam_file)
    print samtools_command, "\n"
    start_time = time.time()
    !$samtools_command
    print "\n* samtools time consume = ", time_consume(start_time), "\n"
    
    #samtools_sort
    sorted_bam_file = unsorted_bam_file.replace(".unsorted", "")
    samtools_sort_command = "%s sort %s -o %s" % (samtools, unsorted_bam_file, sorted_bam_file)
    print samtools_sort_command, "\n"
    start_time = time.time()
    !$samtools_sort_command
    os.remove(unsorted_bam_file)
    print "\n* samtools_sort time consume = ", time_consume(start_time), "\n"

    
    #cufflinks
    cufflinks_out = rna_seq_folder + "/cufflinks/cufflinks_%s" % sample_name
    cufflinks_command = "%s --library-type fr-firststrand -p 4 -G %s %s -o %s " % (
                        cufflinks, gff_file, sorted_bam_file, cufflinks_out)
    print cufflinks_command, "\n"
    start_time = time.time()
    !$cufflinks_command
    print "\n* cufflinks time consume = ", time_consume(start_time), "\n"
    
    #make gff
    gff_file_ = rna_seq_folder + "/gff/%s" % sample_name
    makegff_command = "python %s --separate_strand --log_scale --chromosome_size %s %s %s" % (
                             makegff, chromosome_size, sorted_bam_file, gff_file_)
    print makegff_command, "\n"
    start_time = time.time()
    !$makegff_command
    print "\n* make_gff time consume = ", time_consume(start_time), "\n"
    
    print "\n------------------------------------------------------------------------\n"
    #break

/home/j1lim/sbl_nas/MAP/RNA-seq/sequencing_result/fastq/ATCC19688-1_1.fastq & /home/j1lim/sbl_nas/MAP/RNA-seq/sequencing_result/fastq/ATCC19688-1_2.fastq is now analyzing...

bowtie -X 1000 -n 2 -p 4 -3 3 -S /media/sf_j1/MAP/RNA-seq/analysis/reference/GCF_000007865.1_ASM786v1_genomic.fna -1 /home/j1lim/sbl_nas/MAP/RNA-seq/sequencing_result/fastq/ATCC19688-1_1.fastq -2 /home/j1lim/sbl_nas/MAP/RNA-seq/sequencing_result/fastq/ATCC19688-1_2.fastq > /media/sf_j1/MAP/RNA-seq/analysis/alignment/ATCC19688-1.sam --un /media/sf_j1/MAP/RNA-seq/analysis/unaligned/unaligned_ATCC19688-1 

# reads processed: 17851509
# reads with at least one reported alignment: 16826555 (94.26%)
# reads that failed to align: 1024954 (5.74%)
Reported 16826555 paired-end alignments to 1 output stream(s)

* bowtie time consume =  0 hr 32.0 min 48.1955008507 sec 

/home/j1lim/다운로드/samtools-1.9/samtools view -bS /media/sf_j1/MAP/RNA-seq/analysis/alignment/ATCC19688-1.sam -o /media/sf_j1/MAP/RNA-seq/analysis/alignment/ATC

In [9]:
#mapping results statistic
sam_files = os.listdir(rna_seq_folder + "/alignment/")
sam_files.sort()

for x in sam_files:
    if ".sam" not in x:
        continue
    if "ATCC" not in x:
        continue
    if "_ori." in x:
        continue
    print "*** %s mapping results statistic analyzing...\n" % x
    sam_file_for_stats = rna_seq_folder + "/alignment/%s" % x
    stats_file = "/media/sf_j1/MAP/RNA-seq/analysis/statistic/%s" % x.replace(".sam", "_stat.txt")
    mapping_statistic_command = "%s stats %s > %s" % (samtools, sam_file_for_stats, stats_file)
    print mapping_statistic_command, "\n"
    start_time = time.time()
    !$mapping_statistic_command
    print "\n* mappint result statistic time consume = ", time_consume(start_time), "\n"
    
    print "------------------------------------------------------------------------"

*** ATCC19688-1.sam mapping results statistic analyzing...

/home/j1lim/다운로드/samtools-1.9/samtools stats /media/sf_j1/MAP/RNA-seq/analysis/alignment/ATCC19688-1.sam > /media/sf_j1/MAP/RNA-seq/analysis/statistic/ATCC19688-1_stat.txt 


* mappint result statistic time consume =  0 hr 6.0 min 29.3889729977 sec 

------------------------------------------------------------------------
*** ATCC19688-2.sam mapping results statistic analyzing...

/home/j1lim/다운로드/samtools-1.9/samtools stats /media/sf_j1/MAP/RNA-seq/analysis/alignment/ATCC19688-2.sam > /media/sf_j1/MAP/RNA-seq/analysis/statistic/ATCC19688-2_stat.txt 


* mappint result statistic time consume =  0 hr 5.0 min 8.81126213074 sec 

------------------------------------------------------------------------


In [11]:
#bam file directories
bam_dir = rna_seq_folder + "/alignment/"
bam_files = os.listdir(bam_dir)
bam_files.sort()
for x in bam_files:
    if ".bam" not in x:
        bam_files.remove(x)
for x in bam_files:
    if ".unsorted" in x:
        bam_files.remove(x)
print bam_files

['ATCC19698-1.bam', 'ATCC19698-2.bam']


In [8]:
#cuffdiff
sample1 = bam_dir + bam_files[0]
sample2 = bam_dir + bam_files[1]
cuffdiff_out = rna_seq_folder + "/cuffdiff/cuffdiff_%s_%s" % (
                                bam_files[0].replace(".bam", ""), bam_files[1].replace(".bam", ""))
cuffdiff_command = "%s --library-type fr-firststrand -p 4 %s %s %s -o %s" % (
                    cuffdiff, gff_file, sample1, sample2, cuffdiff_out)
print cuffdiff_command, "\n"
start_time = time.time()
!$cuffdiff_command
print "\n* cuffdiff time consume = ", time_consume(start_time), "\n"

/home/j1lim/다운로드/cufflinks/cuffdiff --library-type fr-firststrand -p 4 /media/sf_j1/AM1/1.genome/am1/GCF_000022685.1_ASM2268v1/GCF_000022685.1_ASM2268v1_genomic.gff /home/j1lim/sbl_nas/rna-seq/alignment/am1_Formate.bam /home/j1lim/sbl_nas/rna-seq/alignment/am1_MeOH.bam -o /home/j1lim/sbl_nas/rna-seq/cuffdiff/cuffdiff_am1_Formate_am1_MeOH 

[10:18:17] Loading reference annotation.
[10:18:18] Inspecting maps and determining fragment length distributions.
[10:31:13] Modeling fragment count overdispersion.
> Map Properties:
>	Normalized Map Mass: 6175199.20
>	Raw Map Mass: 4635659.00
>	Fragment Length Distribution: Empirical (learned)
>	              Estimated Mean: 208.84
>	           Estimated Std Dev: 65.56
> Map Properties:
>	Normalized Map Mass: 6175199.20
>	Raw Map Mass: 6807435.00
>	Fragment Length Distribution: Empirical (learned)
>	              Estimated Mean: 211.80
>	           Estimated Std Dev: 66.50
[10:32:39] Calculating preliminary abundance estimates
[10:32:39] Testing fo